In [ ]:
from azureml.core import Workspace

In [ ]:
workspace = Workspace.from_config()

In [ ]:
workspace

In [ ]:
from azureml.core.model import Model

### Registering the model to the workspace

In [ ]:
#skip if model is already registered
model = Model.register(model_path="./Standing_laying/",
                       model_name="standing_laying_classifier",
                       description="Standing/laying classifier",
                       workspace=workspace)

### If the model is already registered in the workspace fetch it using the model name

In [ ]:
model = Model(workspace, name='standing_laying_classifier')

In [ ]:
model

### Entry script for deploying on a AKS(Azure Kubernetes Service)

In [ ]:
%%writefile score_aks.py
import os
import sys

from tensorflow.keras.models import load_model

sys.path.append('/structure/azureml-app/azureml-models/standing_laying_classifier/4/')  # noqa:E402
from Standing_laying.utils import process_img  # noqa:E402


def init():
    global model
    output_directory = '/structure/azureml-app/azureml-models/standing_laying_classifier/4/Standing_laying'
    model = load_model(os.path.join(output_directory, "best_model.h5"))
    print(model)


def run(file):
    try:
        img = process_img(file)
        result = model.predict(img)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Entry script to deploy to ACI(Azure Container Instance)

In [ ]:
%%writefile score_aci.py
import os
import sys

from tensorflow.keras.models import load_model

sys.path.append('/structure/azureml-app/azureml-models/standing_laying_classifier/4/')  # noqa:E402
from Standing_laying.utils import process_img  # noqa:E402


def init():
    global model
    output_directory = '/structure/azureml-app/azureml-models/standing_laying_classifier/4/Standing_laying'
    model = load_model(os.path.join(output_directory, "best_model.h5"))
    print(model)


def run(file):
    try:
        img = process_img(file)
        result = model.predict(img)
        # You can return any data type, as long as it is JSON serializable.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

### Define the dependencies required for the model and set inference config

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment('Standing_laying')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow==2.0.0'],pip_packages=['azureml-defaults'])
inference_config_aci = InferenceConfig(entry_script="score_aci.py", environment=env)
inference_config_aks = InferenceConfig(entry_script="score_aks.py", environment=env)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4)
service = Model.deploy(workspace, "aci-standing-laying-v", [model], inference_config_aci, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
service.update(enable_app_insights=True)

In [ ]:
service.get_logs()

## Creating Inference cluster

For provisioning_configuration(), if you pick custom values for agent_count and vm_size, and cluster_purpose is not DEV_TEST, then you need to make sure agent_count multiplied by vm_size is greater than or equal to 12 virtual CPUs. For example, if you use a vm_size of "Standard_D3_v2", which has 4 virtual CPUs, then you should pick an agent_count of 3 or greater.

### skip if already created

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
#prov_config = AksCompute.provisioning_configuration()

aks_name = 'AKS-interference'

prov_config = prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_A4_v2",
                                                       agent_count = 3,
                                                       location = "westeurope")

# Create the cluster
aks_target = ComputeTarget.create(workspace=workspace,
                                  name=aks_name,
                                  provisioning_configuration=prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice

### Deploy the model on created AKS

In [ ]:
#deployment_config = AksWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 8, autoscale_enabled=True, autoscale_max_replicas=3, collect_model_data=True, enable_app_insights=True)

In [ ]:
model

In [ ]:
workspace

In [ ]:
aks_target = AksCompute(workspace, "AKS-interference")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 2, collect_model_data=True, enable_app_insights=True)
service = Model.deploy(workspace, "aks-standing-laying", [model], inference_config_aks, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
service.get_logs()